# MQT Predictor Usage

In [ ]:
from mqt.predictor import qcompile
from mqt.bench import get_benchmark

## Generated Uncompiled Quantum Circuit

In [ ]:
qc = get_benchmark("ghz", level="indep", circuit_size=5)
qc.draw()

In [ ]:
qc_compiled, compilation_information, quantum_device = qcompile(qc)

## Predicted Device

In [ ]:
quantum_device

## Top 3 Bins

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xlim(bins[0], bins[-1])

## Calcualte Data Points

In [ ]:
# def calculate_x_highest(row, column_name, index):
#     columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
#     values = sorted([value for col, value in row[columns].items() if value != -1], reverse=True)
#     return values[index-1] if len(values) >= 3 else 0

def calculate_median(row, column_name):
    columns = [col for col in row.index if col.endswith(column_name) and not "MQT" in col]
    values = [value for col, value in row[columns].items() if value != -1]
    return np.median(values) if values else None


df['fidelity_median'] = df.apply(calculate_median, column_name="_fidelity",axis=1)
df['critical_depth_median'] = df.apply(calculate_median, column_name="_critical_depth",axis=1)

In [ ]:
df['fidelity_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['fidelity_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].max(), axis=1)
df['critical_depth_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['critical_depth_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].max(), axis=1)

## Normalize

In [ ]:
# divider_column_name = 'fidelity_max_other'

# # Get the list of column names ending with "_fidelity"
# columns_to_divide = [col for col in df.columns if col.endswith("_fidelity") or col in ['fidelity_min_other', 'fidelity_max_other', 'fidelity_median' ]]

# # Iterate through each column and perform the division
# for col_name in columns_to_divide:
#     df[col_name] = df[col_name].divide(df[divider_column_name])

# divider_column_name = 'critical_depth_max_other'

# # Get the list of column names ending with "_fidelity"
# columns_to_divide = [col for col in df.columns if col.endswith("_critical_depth") or col in ['critical_depth_min_other', 'critical_depth_max_other',  'critical_depth_median' ]]

# # Iterate through each column and perform the division
# for col_name in columns_to_divide:
#     df[col_name] = df[col_name].divide(df[divider_column_name])

In [ ]:
print("Fid Improvement: ", df_filtered_fidelity.loc[df['MQTPredictor_fidelity_fidelity'] != np.inf, 'MQTPredictor_fidelity_fidelity'].max())

In [ ]:
len(df_filtered_fidelity)

In [ ]:
df_filtered_fidelity['MQTPredictor_fidelity_fidelity'].describe()

In [ ]:
plot_kind = "line"
df_filtered_and_sorted_fidelity = df_filtered_fidelity.sort_values(by=['MQTPredictor_fidelity_fidelity'])
ax = df_filtered_and_sorted_fidelity.plot(x="file_path", y="fidelity_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df_filtered_and_sorted_fidelity.plot(x="file_path", y="MQTPredictor_fidelity_fidelity", label="MQT Predictor", kind=plot_kind, rot=90, ax=ax, color="blue")
df_filtered_and_sorted_fidelity.plot(x="file_path", y="fidelity_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")
df_filtered_and_sorted_fidelity.plot(x="file_path", y="fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")

if plot_kind == "line":
    plt.xticks(range(len(df_filtered_and_sorted_fidelity.file_path)), df_filtered_and_sorted_fidelity.file_path)

plt.xticks(
    list(range(0, len(df_filtered_and_sorted_fidelity.file_path), 1)),
    [ "" for i in range(0, len(df_filtered_and_sorted_fidelity.file_path), 1)],
    fontsize=16,
)
#df_filtered_and_sorted_fidelity.iloc[i]["num_qubits"]


plt.xlabel("Benchmarks")
plt.ylabel("Expected Fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 

plt.savefig("results/fidelity_plot.pdf", bbox_inches='tight')

## Top 3 Fidelity

In [ ]:
df_filtered_and_sorted_fidelity['Rank_MQT_fidelity'] = df_filtered_and_sorted_fidelity.apply(lambda row: sum(1 for col in df_filtered_and_sorted_fidelity.columns if col.endswith('_fidelity') and not "MQT" in col and row['MQTPredictor_fidelity_fidelity'] >=row[col]), axis=1)
df_filtered_and_sorted_fidelity['Rank_MQT_fidelity'] = len([col for col in df_filtered_and_sorted_fidelity.columns if col.endswith('_fidelity') and not "MQT" in col]) - df_filtered_and_sorted_fidelity.Rank_MQT_fidelity
plt.hist(df_filtered_and_sorted_fidelity.Rank_MQT_fidelity.values, bins=range(0,15,1), align="left", weights=np.ones(len(df_filtered_and_sorted_fidelity)) / len(df_filtered_and_sorted_fidelity))
plt.xticks(range(0,14,1));

In [ ]:
len(df_filtered_and_sorted_fidelity[df_filtered_and_sorted_fidelity['Rank_MQT_fidelity']==0])

In [ ]:
len(df_filtered_and_sorted_fidelity)

In [ ]:
count_top3 = 0
for index, row in df_filtered_and_sorted_fidelity.iterrows():
    if row['Rank_MQT_fidelity'] in [0, 1, 2]:
        count_top3 += 1
        
print("Percentage of Top-3:", count_top3/len(df_filtered_and_sorted_fidelity))

In [ ]:
print("Crit. Dep. Improvement: ", df_filtered_critical_depth.loc[df['MQTPredictor_critical_depth_critical_depth'] != np.inf, 'MQTPredictor_critical_depth_critical_depth'].max())

In [ ]:
len(df_filtered_critical_depth)

In [ ]:
plot_kind = "line"
df_filtered_and_sorted_critical_depth = df_filtered_critical_depth.sort_values(by=['MQTPredictor_critical_depth_critical_depth'])
ax = df_filtered_and_sorted_critical_depth.plot(x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(30,10))
df_filtered_and_sorted_critical_depth.plot(x="file_path", y="MQTPredictor_critical_depth_critical_depth", kind=plot_kind, rot=90, ax=ax, color="blue", label="MQT Predictor")
df_filtered_and_sorted_critical_depth.plot(x="file_path", y="critical_depth_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")
df_filtered_and_sorted_critical_depth.plot(x="file_path", y="critical_depth_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")


if plot_kind == "line":
    plt.xticks(range(len(df_filtered_and_sorted_critical_depth.file_path)), df_filtered_and_sorted_critical_depth.file_path);


plt.xticks(
    list(range(0, len(df_filtered_and_sorted_critical_depth.file_path), 1)),
    ["" for i in range(0, len(df_filtered_and_sorted_critical_depth), 1)],
    fontsize=18,
)
#df_filtered_and_sorted_critical_depth.iloc[i]["num_qubits"]
plt.xlabel("Benchmarks")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 

plt.savefig("results/critical_depth_plot.pdf", bbox_inches='tight')

## Top 3 Critical Depth

In [ ]:
df_filtered_and_sorted_critical_depth['Rank_MQT_critical_depth'] = df_filtered_and_sorted_critical_depth.apply(lambda row: sum(1 for col in df_filtered_and_sorted_critical_depth.columns if col.endswith('_critical_depth') and not "MQT" in col and row['MQTPredictor_critical_depth_critical_depth'] >= row[col]), axis=1)
df_filtered_and_sorted_critical_depth['Rank_MQT_critical_depth'] = len([col for col in df_filtered_and_sorted_critical_depth.columns if col.endswith('_critical_depth') and not "MQT" in col]) - df_filtered_and_sorted_critical_depth.Rank_MQT_critical_depth
plt.hist(df_filtered_and_sorted_critical_depth.Rank_MQT_critical_depth.values, bins=range(0,15,1), align="left", weights=np.ones(len(df_filtered_and_sorted_critical_depth)) / len(df_filtered_and_sorted_critical_depth))
plt.xticks(range(0,14,1));

In [ ]:
len(df_filtered_and_sorted_critical_depth[df_filtered_and_sorted_critical_depth['Rank_MQT_critical_depth']==0])

In [ ]:
len(df_filtered_and_sorted_critical_depth)

In [ ]:
count_top3 = 0
for index, row in df_filtered_and_sorted_critical_depth.iterrows():
    if row['Rank_MQT_critical_depth'] in [0, 1, 2]:
        count_top3 += 1

# Print the total count
print("Percentage of Top-3:", count_top3/len(df_filtered_and_sorted_critical_depth))

In [ ]:
df = pd.read_csv(ml.helper.get_path_trained_model() / 'res_GHZ.csv', sep=',')
df = df[df.num_qubits<32]
plt.rcParams["font.size"] = 50

In [ ]:
# df['fidelity_second_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=2, axis=1)
# df['fidelity_third_highest'] = df.apply(calculate_x_highest, column_name="_fidelity", index=3, axis=1)
df['fidelity_median'] = df.apply(calculate_median, column_name="_fidelity",axis=1)


# df['critical_depth_third_highest'] = df.apply(calculate_x_highest, column_name="_critical_depth", index=3,axis=1)
df['critical_depth_median'] = df.apply(calculate_median, column_name="_critical_depth",axis=1)

df['fidelity_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['fidelity_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_fidelity') and 'MQT' not in col]].max(), axis=1)
df['critical_depth_min_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].replace(-1, np.nan).min(skipna=True), axis=1)
df['critical_depth_max_other'] = df.apply(lambda row: row[[col for col in row.index if col.endswith('_critical_depth') and 'MQT' not in col]].max(), axis=1)


In [ ]:
plot_kind = "line"
df = df.sort_values(by=['num_qubits'])
ax = df.plot(x="file_path", y="fidelity_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(20,10))
df.plot(x="file_path", y="MQTPredictor_fidelity_fidelity", label="MQT Predictor", kind=plot_kind, rot=90, ax=ax, color="blue")
df.plot(x="file_path", y="fidelity_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")
df.plot(x="file_path", y="fidelity_min_other", label="Worst", rot=90, kind=plot_kind, ax=ax, color="red")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path)

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Expected Fidelity")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2,3]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 


plt.savefig("results/fidelity_ghz_plot.pdf", bbox_inches='tight')

In [ ]:
plot_kind = "line"
df = df.sort_values(by=['num_qubits'])
ax = df.plot(x="file_path", y="critical_depth_max_other", label="Best", rot=90, kind=plot_kind, color="green", figsize=(20,10))
df.plot(x="file_path", y="MQTPredictor_critical_depth_critical_depth", kind=plot_kind, rot=90, ax=ax, color="blue", label="MQT Predictor")
df.plot(x="file_path", y="critical_depth_median", kind=plot_kind, rot=90, ax=ax, color="orange", label="Median")

if plot_kind == "line":
    plt.xticks(range(len(df.file_path)), df.file_path);

plt.xticks(
    list(range(0, len(df), 1)),
    [df.iloc[i].num_qubits if i % 4 == 1 else "" for i in range(len(df))],
)

plt.xlabel("Number of Qubits")
plt.ylabel("Critical Depth")

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,0,2]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order]) 

plt.savefig("results/critical_depth_ghz_plot.pdf", bbox_inches='tight')

## Predicted and Conducted Compilation Passes

In [ ]:
compilation_information

## Compiled Circuit

In [ ]:
qc_compiled.draw()